In [1]:
import pandas as pd

# AIMS

1. tasa de morosidad
2. Tasa de incumplimiento

## 1. tasa de morosidad

In [2]:
prestamos = pd.read_csv('prestamos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
prestamos.columns = ['id_prestamo', 'monto', 'fecha_inicio', 'fecha2', 'fecha_pago','dias_atraso']

prestamos['id_prestamo'] = prestamos['id_prestamo'].astype(str)
prestamos['fecha_inicio'] = pd.to_datetime(prestamos['fecha_inicio'], dayfirst='true', errors = 'coerce')
prestamos['fecha2'] = pd.to_datetime(prestamos['fecha2'], dayfirst='true', errors = 'coerce')
prestamos['fecha_pago'] = pd.to_datetime(prestamos['fecha_pago'], dayfirst='true', errors = 'coerce')

prestamos.head()

,id_prestamo,monto,fecha_inicio,fecha2,fecha_pago,dias_atraso
0,10001,100,2022-12-12,2022-12-12,2022-12-12,0
1,10002,200,2022-12-22,2022-12-22,2022-12-23,1
2,10003,50,2022-12-23,2022-12-23,2022-12-28,5
3,10004,75,2022-12-24,2022-12-24,2022-12-28,4
4,10005,125,2022-12-24,2022-12-24,2022-12-29,5


In [3]:
prestamos_agrupados = prestamos.groupby('fecha_inicio')['monto'].sum()
prestamos_agrupados

fecha_inicio
2022-12-12    100
2022-12-22    200
2022-12-23     50
2022-12-24    500
2022-12-25    450
Name: monto, dtype: int64

In [4]:
atrasos_agrupados = prestamos.groupby('fecha_inicio')['dias_atraso'].sum()
atrasos_agrupados

fecha_inicio
2022-12-12    0
2022-12-22    1
2022-12-23    5
2022-12-24    9
2022-12-25    7
Name: dias_atraso, dtype: int64

In [5]:
morosidad = pd.concat([prestamos_agrupados, atrasos_agrupados], axis=1)
morosidad['tasa'] = morosidad['dias_atraso'] / morosidad['monto']
morosidad

,monto,dias_atraso,tasa
fecha_inicio,,,
2022-12-12,100,0,0.000000
2022-12-22,200,1,0.005000
2022-12-23,50,5,0.100000
2022-12-24,500,9,0.018000
2022-12-25,450,7,0.015556


## 2. Tasa de incumplimiento

In [11]:
creditos = pd.DataFrame({
    "Cliente": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro", 'Carlos'],
    "Monto": [1000, 5000, 2000, 3000, 1500, 2000, 800],
    "Cuotas": [12, 24, 6, 12, 8, 12, 10],
    "Fecha": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01", "2022-02-01"],
    "dias_mora":[0,0,25,0,11,0,9]
})
creditos["Fecha"] = pd.to_datetime(creditos["Fecha"])

creditos.insert(5, 'count', 1)

creditos.head()

,Cliente,Monto,Cuotas,Fecha,dias_mora,count
0,Juan,1000,12,2022-01-01,0,1
1,Juan,5000,24,2022-02-01,0,1
2,Ana,2000,6,2022-01-01,25,1
3,Ana,3000,12,2022-03-01,0,1
4,Pedro,1500,8,2022-01-01,11,1


In [18]:
ndf = creditos.groupby('Cliente')['count'].sum().reset_index()
ndf.rename(columns={'count':'credits_total'}, inplace=True)

ndf2 = creditos[creditos['dias_mora'] == 0 ]
ndf2 = ndf2.groupby('Cliente')['count'].sum().reset_index()
ndf2.rename(columns={'count':'credits_mora'}, inplace=True)

ndf = pd.merge(ndf, ndf2, how='left', on=['Cliente'])
ndf = ndf.fillna(0)  

ndf['mora_tasa'] = ndf['credits_mora'] / ndf['credits_total']

ndf

,Cliente,credits_total,credits_mora,mora_tasa
0,Ana,2,1.0,0.5
1,Carlos,1,0.0,0.0
2,Juan,2,2.0,1.0
3,Pedro,2,1.0,0.5
